In [1]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score
from datetime import datetime

# --- Début du Fichier main.py ---

def load_data(file_path='../Data/processed/sirene_infos_CLEAN.parquet'):
    """Charge les données et sépare X et y."""
    try:
        df = pd.read_parquet(file_path)
    except FileNotFoundError:
        print(f"Erreur: Le fichier {file_path} est introuvable.")
        return None, None, None

    # 1. Définition de la variable cible (y)
    y = df['is_failed_in_3y']

    # 2. Définition des features (X) et exclusion des colonnes de fuite
    leakage_columns = ['is_failed_in_3y', 'dateFermeture', 'date_limite_3_ans', 'siren']
    X = df.drop(columns=leakage_columns, errors='ignore')

    # Conversion de la date au bon format si ce n'est pas déjà fait
    if 'dateCreationUniteLegale' in X.columns:
        X['dateCreationUniteLegale'] = pd.to_datetime(X['dateCreationUniteLegale'])

    return X, y, df

# X, y, df = load_data()

In [2]:
def temporal_split(X, y, split_date='2018-01-01'):
    """Effectue un split d'entraînement et de test basé sur le temps."""
    
    split_dt = pd.to_datetime(split_date)
    
    # Séparation : Train = avant la date, Test = à partir de la date
    train_mask = X['dateCreationUniteLegale'] < split_dt
    
    X_train = X[train_mask].copy()
    y_train = y[train_mask].copy()
    
    X_test = X[~train_mask].copy()
    y_test = y[~train_mask].copy()
    
    print(f"\n--- Split Temporel Effectué (Coupure: {split_date}) ---")
    print(f"Taille du Train: {len(X_train)} entreprises.")
    print(f"Taille du Test: {len(X_test)} entreprises.")
    print(f"Proportion de défaillances (1) dans Train: {y_train.mean():.2%}")
    print(f"Proportion de défaillances (1) dans Test: {y_test.mean():.2%}")
    
    # Retirer la date de création de X_train et X_test après le split
    # (Elle est maintenant redondante avec anneeCreation/moisCreation)
    X_train = X_train.drop(columns=['dateCreationUniteLegale'])
    X_test = X_test.drop(columns=['dateCreationUniteLegale'])

    return X_train, X_test, y_train, y_test

In [3]:
def build_preprocessor(X_train):
    """Définit le pré-processing pour les différents types de colonnes."""
    
    # Variables Numériques (anneeCreation, moisCreation)
    numeric_features = ['anneeCreation', 'moisCreation']
    
    # Variables Catégorielles (toutes les autres après retrait de numeric/date)
    # On enlève toutes les colonnes numériques/temporelles restantes
    categorical_features = [col for col in X_train.columns if col not in numeric_features]

    # Pipeline pour les variables numériques
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler()) # Standardisation des variables
    ])
    
    # Pipeline pour les variables catégorielles (One-Hot Encoding)
    # handle_unknown='ignore' permet de gérer les nouvelles catégories dans le test set
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])
    
    # Combinaison des transformations
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='drop' # Supprime toute colonne non mentionnée
    )
    
    return preprocessor

# preprocessor = build_preprocessor(X_train)

In [4]:
def build_baseline_pipeline(preprocessor):
    """Construit et retourne le pipeline complet de baseline (Log. Reg.)."""
    
    # Le modèle baseline : Régression Logistique avec gestion du déséquilibre
    # class_weight='balanced' ajuste automatiquement les poids
    # proportionnellement aux fréquences des classes dans les données d'entraînement.
    baseline_model = LogisticRegression(
        random_state=42, 
        solver='liblinear', 
        class_weight='balanced'
    )
    
    # Pipeline complet : Pré-processing -> Modèle
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', baseline_model)
    ])
    
    return pipeline

# pipeline = build_baseline_pipeline(preprocessor)

In [5]:
def train_and_evaluate(pipeline, X_train, X_test, y_train, y_test):
    """Entraîne le modèle et évalue les métriques sur le test set."""
    
    print("\n--- Entraînement du Modèle Baseline ---")
    pipeline.fit(X_train, y_train)
    
    print("--- Évaluation des Métriques sur le Test Set ---")
    
    # Prédictions et Probabilités
    y_pred = pipeline.predict(X_test)
    # Nous utilisons predict_proba pour la métrique ROC AUC
    y_proba = pipeline.predict_proba(X_test)[:, 1] # Probabilité de la classe 1 (faillite)
    
    # Calcul des métriques
    metrics = {
        "ROC AUC": roc_auc_score(y_test, y_proba),
        "Recall (Rappel)": recall_score(y_test, y_pred), # Important pour ne pas rater les défaillances
        "Precision (Précision)": precision_score(y_test, y_pred, zero_division=0),
        "F1-Score": f1_score(y_test, y_pred),
    }

    print("\n✅ Résultats du Baseline sur l'Ensemble de Test:")
    for name, value in metrics.items():
        print(f"   * {name}: {value:.4f}")
        
    return metrics